In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import os

from scipy.stats import hmean

from functools import reduce
from numpy.typing import ArrayLike
import warnings
import openpyxl

import sys
sys.path.append("../../") # noqa

from typing import Generator, Tuple

from python_src.figures_utils import get_all_expected, generate_experimental_df, get_relabund_files, fully_combined

# NIST Sample Sensitivity Analysis

Our goal is to use the five NIST samples to conduct an analysis of TP, TN, FP, and FN scores across the bacterial species.

In [ ]:
# We should read in the expected data and make a large dataframe.

expected_df = get_all_expected("../../expected_pipelines/nist/", rank="species")
expected_df.drop(columns=["Source"], inplace=True)
display(expected_df)

expected_df.reset_index(inplace=True)
df = expected_df.pivot(index=["species", "TAX_ID"], columns="SampleID", values="RA")
df

In [ ]:
fc = fully_combined(root_dir="../../pipelines/nist", expected_root="../../expected_pipelines/nist", rank="species")
display(fc)

negatives = get_relabund_files("negatives", rank="species")
display(negatives)

In [ ]:
def make_dataframes():
    for smpl, smpl_df in fc.groupby("SampleID"):
        exp = smpl_df.loc[smpl_df["Source"] == "Expected"]
        obs = smpl_df.loc[smpl_df["Source"] != "Expected"]

        # Take from the expected dataframe the index, species, and RA.
        final_df = pd.DataFrame()
        final_df.index = exp.index
        final_df["species"] = exp["species"]
        final_df["RA_exp"] = exp["RA"]

        data_frames = smpl_df.groupby("Source")
        for pl, pl_df in obs.groupby("Source"):
            # left join with exp
            merged = exp.merge(pl_df, on="TAX_ID", how="left", suffixes=("_exp", "_obs"))

            # Don't need most of these columns, want final table to be TAXID, Species Name, SampleID_exp, BB3, BB4, etc.
            merged.drop(columns=["Source_exp", "Source_obs", "species_obs", "SampleID_exp", "SampleID_obs"], inplace=True)

            # Rename the columns to be SampleID_exp, SampleID_obs
            merged = merged.rename(columns={"RA_exp": f"{smpl}_exp", "RA_obs": f"{pl}_obs"})

            final_df = pd.concat([final_df, merged[f"{pl}_obs"]], axis=1, sort=False)

        final_df.index.name = "TAX_ID"
        final_df["SampleID"] = smpl
        # display(final_df.head())
        yield final_df

"""
def calculate_stats(df: pd.DataFrame, pl: str):
    df = df.copy()
    df.fillna(0, inplace=True)
    # TP condition: species is in expected and observed = 0
    # FP condition: species is in observed but not expected = 1
    # FN condition: species is in expected but not observed = 2
    # TN condition: species is not in expected or observed = 3

    conditions = [
        (df["RA_exp"] > 0) & (df[f"{pl}_obs"] > 0),
        (df["RA_exp"] == 0) & (df[f"{pl}_obs"] > 0),
        (df["RA_exp"] > 0) & (df[f"{pl}_obs"] == 0),
        (df["RA_exp"] == 0) & (df[f"{pl}_obs"] == 0),
    ]

    choices = [0, 1, 2, 3]

    df["score"] = np.select(conditions, choices)

    # Now, summarize the number of each condition

    summary = df["score"].value_counts()
    print(summary)

    display(df.head()) 
"""

def pivot_df(dataframes: Generator[pd.DataFrame, None, None]) -> Generator[Tuple[pd.DataFrame, str], None, None]:
    """
    Takes the dataframes from make_dataframes and pivots them so that the SampleID values are the columns.

    Parameters:
        dataframes: Generator[pd.DataFrame]
            The dataframes from make_dataframes
    
    Returns:
        pd.DataFrame as a generator
    
    """
    pipelines = ["biobakery3", "biobakery4", "jams", "wgsa2", "woltka"]

    for pl in pipelines:
        merged = pd.DataFrame()
        for dfs in make_dataframes():
            pl_df = dfs[["species", "RA_exp", f"{pl}_obs", "SampleID"]]
            merged = pd.concat([merged, pl_df], axis=0, sort=False)
            # calculate_stats(pl_df, pl)

        # display(merged.head())
        # print(merged.shape)

        # Make the sampleID values the columns
        print(f"Pipeline: {pl}")
        piv = merged.pivot(index=["species", "RA_exp"], columns="SampleID", values=f"{pl}_obs")
        display(piv)

        yield (piv, pl)

In [ ]:
# We have several conditions. 
# One set is if both values are above the treshold. If so, we can check if:
# 1. Both are above the threshold (TP)
# 2. Both are zero. (TN)
# 3. Observed is above the threshold and the expected is zero. (FP)
# 4. Observed is below the threshold and the expected is above the threshold. (FN)

# Unfortunately, sometimes the expected is below the threshold. In this case, we need to relax the observed threshold to above 0.
# Thus, a TP would be if the observed is above 0 and the expected is above 0. A TN would be if the observed is 0 and the expected is 0.

def confusion_matrix(df: pd.DataFrame, threshold: float):
    output_list = []
    for spec, spec_df in df.groupby("species"):
        # Initialize the confusion matrix
        tp, fp, fn, tn = 0, 0, 0, 0

        for ind in spec_df.index:
            ra_exp = ind[1]
            # display(spec_df.loc[[ind]])

            # Get index of True value in the row
            row_values = spec_df.loc[[ind]].values.reshape(-1)
            truth_array = spec_df.loc[[ind]].notna().values.reshape(-1)
            # print(truth_array)
            idx = [i for i, x in enumerate(truth_array) if x]

            if len(idx) > 1:
                raise Exception("More than one value in row")
            
            # Means that the pipeline did not find the value at all. This can either be a true negative or a false negative.
            ra_obs: float = None

            try:
                # display(spec, spec_df.loc[[ind]].values.reshape(-1))
                ra_obs = spec_df.loc[[ind]].values.reshape(-1)[idx][0]
            except IndexError:
                print("Could not find spec: ", spec)
                # If we can't find the value, then the pipeline did not find it.
                ra_obs = 0

            # If the value is less than the threshold, it is not considered to be present. 
            
            # True positive
            if ra_exp > threshold and ra_obs > threshold:
                tp += 1
            # False negative
            elif ra_exp > threshold and ra_obs <= threshold:
                fn += 1
            # False positive
            elif ra_exp == 0 and ra_obs > threshold:
                fp += 1
            # True negative
            elif ra_exp == 0 and ra_obs <= threshold:
                tn += 1
            else:
                if ra_exp < threshold and ra_exp != 0.0:
                    # This warning is necessary because sometimes the expected value is below the threshold. If so, the observed
                    # threshold needs to be relaxed to some other value.
                    warnings.warn(f"WARN: Expected value for {spec} is below threshold of {threshold} with value: {ra_exp}. The filtering criteria for the observed value will be relaxed to above 0 for a TP.")
                    # If the expected value is below the threshold, we need to relax the observed threshold to above 0.
                    if ra_obs > 0:
                        tp += 1
                    # If the expected value is then zero, we have a false negative.
                    if ra_obs == 0:
                        fn += 1
                else:
                    raise Exception(f"Unexpected condition: {ra_exp}, {ra_obs}")

        # Verify that the sum is five since we have five samples.
        if tp + fp + fn + tn != 5:
            print(f"Results for {spec}: {tp}, {fp}, {fn}, {tn}")
            raise Exception("Sum of confusion matrix is not 5")

        # print(f"Results for {spec}: {tp}, {fp}, {fn}, {tn}")
        output_list.append([spec, tp, fp, fn, tn])

    output_df = pd.DataFrame(output_list, columns=["species", "tp", "fp", "fn", "tn"])
    return output_df

def performance_metrics(df: pd.DataFrame, pipeline: str):
    # We need to calculate the following metrics:
    # Sens = TP / (TP + FN)
    # Spec = TN / (TN + FP)
    # Prec = TP / (TP + FP)
    # Acc = (TP + TN) / (TP + TN + FP + FN)
    # F1 = 2 * (Prec * Sens) / (Prec + Sens)
    # DOR = Sens * Spec / ((1 - Sens) * (1 - Spec))

    def ignore_nan_hmean(x: ArrayLike) -> float:
        # Drop NaN values
        x = x[~np.isnan(x)]
        # Then, compute hmean.
        return hmean(x)

    df["sens"] = df["tp"] / (df["tp"] + df["fn"])
    df["spec"] = df["tn"] / (df["tn"] + df["fp"])
    df["prec"] = df["tp"] / (df["tp"] + df["fp"])
    df["acc"] = (df["tp"] + df["tn"]) / (df["tp"] + df["tn"] + df["fp"] + df["fn"])
    df["f1"] = 2 * (df["prec"] * df["sens"]) / (df["prec"] + df["sens"])
    # df["dor"] = df["sens"] * df["spec"] / ((1 - df["sens"]) * (1 - df["spec"]))

    # We want to calculate the harmonic mean of each column. 
    # Unfortunately, there can be NaN values, so we need to ignore those if present in a column.
    df.loc["Harmonic Mean"] = df[["sens", "spec", "prec", "acc", "f1"]].apply(lambda x: ignore_nan_hmean(x), axis=0)
    df.loc["Mean"] = df[["sens", "spec", "prec", "acc", "f1"]].mean(axis=0, skipna=True)

    return df

In [ ]:
def main(threshold: float):
    dfs = {}
    for piv in pivot_df(make_dataframes()):
        pipeline_name = piv[1]
        cf_df = confusion_matrix(piv[0], threshold)
        cf_df.set_index("species", inplace=True)
        cf_df = performance_metrics(cf_df, pipeline_name)
        cf_df["Threshold"] = threshold

        display(cf_df)

        dfs[pipeline_name] = cf_df

    with pd.ExcelWriter(f"confusion_matrix_{threshold}.xlsx") as writer:
        for key in dfs.keys():
            data: pd.DataFrame = dfs[key]

            # Drop index name and make it a column.
            data.reset_index(inplace=True)

            data.to_excel(writer, sheet_name=key)

            tex_output = os.path.join("tex", str(threshold), f"confusion_matrix_{key}.tex")
            tex_caption = f"Confusion matrix for {key} at threshold {threshold}."
            tex_label = f"tab:conf_{key}_{threshold}"

            # We need to split the species names into two words on the underscore.
            data["species"] = data["species"].str.split("_").str.join(" ")

            # Then, capitalize the first letter of each word.
            data["species"] = data["species"].str.title()

            # All of the index names should be in italics. We should use the CSS styler to do this.
            # data.index = data.index.to_frame().style.set_properties(**{"text-align": "left", "font-style": "italic"}).render()
            s = data.style.format_index(lambda x: f"\\textbf{{{x}}}", axis=1)

            # We want a rule between the header and the data.
            s.set_table_styles([
                {"selector": 'midrule', 'props': ':hline;'},
                {"selector": 'toprule', 'props': ':hline;'},
            ])

            # s = data.style.format(lambda x: f"\\textit{{{x}}}", subset=["species"])
            s.format("{:.0f}", subset=["tp", "fp", "fn", "tn"], na_rep="-")
            s.format("{:.3f}", subset=["sens", "spec", "prec", "acc", "f1"], na_rep="-")
            s.format("{:.4f}", subset=["Threshold"], na_rep="-")

            # We need to unitalicize and bold the Harmonic Mean and Mean rows. These are always the last two rows.
            # s.set_properties(**{"text-align": "left", "font-style": "normal", "font-weight": "bold"}, subset=pd.IndexSlice[-2:, :])
            def bold_means(v):
                if v == "Harmonic Mean" or v == "Mean":
                    return "font-weight: bold; font-style: normal;"
                else:
                    return "font-style: italic;"

            s.applymap(bold_means, subset=["species"])

            s.hide(axis="index")

            # Replace NaN values with a dash.
            # Make the table small size.

            display(data)

            s.to_latex(tex_output, position_float="centering", convert_css=True, position="H")

            with open(tex_output, "r+") as f:
                # text = f.read().replace("\\begin{tabular}", "\small\n\\begin{tabular}")
                replacement_caption = f"\\end{{tabular}}\n\\caption{{{tex_caption}}}\n\\label{{{tex_label}}}"
                caption = f.read().replace("\\end{tabular}", replacement_caption)
                # f.seek(0)
                # f.write(text)
                f.seek(0)
                f.write(caption)
                f.close()

main(0.0)
main(1E-4)